# 프롬프트 엔지니어링 기법들
## Zero-Shot Prompting
* 특정 작업에 대해 명시적으로 훈련받지 않은 상태에서, 모델이 주어진 프롬프트를 통해 바로 작업을 수행하도록 유도하는 것입니다. 이는 대규모 언어 모델의 강력한 일반화 능력을 활용하여 새로운 문제에 대한 해결책을 제시하는 방법

* ChatGPT 이전에는 특정 태스크를 수행하기 위해 데이터 학습이 필요했습니다. 그러나 ChatGPT 3.5, GPT-4, Claude 3 같은 대규모 언어 모델은 추가 학습이나 예시 없이도 상당히 잘 답변할 수 있습니다. 2022년 ChatGPT 출시 이후, 2024년 기준으로 ChatGPT에게 바로 질문하는 것은 자연스러운 행위가 되었습니다.

## 대표적인 기법 Few Shot, Chain of Thought
* 현재 Chain of Thought 기법을 많이 씀
* 어느 기법이 좋다는 것을 없고, 각 기법을 잘 이해하고 원리를 아는게 중요

## 1. Few Shot
* 답변을 얻기 위해 참고할 수 있는 정답 사례를 알려줘 프롬프트 기능을 올려주는 방법
* Q1 : ~~~~? (참고)
* A1 : ~~~ (참고)
* Q2 : ~~~?
* A2 :
* 관련 논문 : https://arxiv.org/abs/2005.14165 (=GPT-3 논문)
* 3-shot의 경우 정답 사례를 프롬프트에 3개 입력
* 공인된 프롬프트 엔지니어링 방법론, 평가에서도 사용, 모델 사이즈의 크기가 어느정도 있어야 함

### 1-1. Few Shot이 잘 동작하는 이유
* 추가학습없이 정답 사례만 추가해도 성능이 급격히 증가 (파라미터를 수정하지 않아도)
* 사전훈련단계에서 잘 훈련된 언어모델
* 태스크에 빠르게 적응
* In-Context Learning이라고 부름

In [1]:
# OpenAI API 호출
from openai import OpenAI

OPENAI_API_KEY = "API KEY를 입력하세요"

# client 호출
client = OpenAI(api_key=OPENAI_API_KEY)

### 1-2. Few Shot example

* 기본적인 프롬프트

In [2]:
prompt = """Q: who wrote the book The Little Prince?"""

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

A: The Little Prince was written by Antoine de Saint-Exupéry.


* 1-shot

In [3]:
prompt = """Answer these questions:
Q: Who sang billie jean?
A: Michale Jackson
Q: Who wrote the book The Little Prince?
A: """

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Antoine de Saint-Exupéry


위 2가지 코드를 비교해보면 1-shot에서 Q에 대한 A는 'Michael Jackson' 한 단어만 쓴 사례를 보여주었고, Antoine de Saint-Exupéry라는 한 단어만 답변을 얻었다.

### 1-3. Few Shot 장점, 단점
장점
* 예시만 넣어주면 되기 때문에 간단하다.

단점
* 예시가 긴 경우, 속도와 비용 영향이 있다.

## 2. Chain-of-Thought
* 정답 사례 + 문제 해결 과정도 Prompt에 추가
* 답을 추리하는 과정을 적어준다.
* 대부분 프롬프트 엔지니어링 기법이 Chain-of-Thought


### Standard Prompting gpt-3.5

In [5]:
# Prompt 출처: https://github.com/microsoft/generative-ai-for-beginners/tree/main/05-advanced-prompts

prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Alice would have 5 apples - 3 thrown + 2 given to Bob - 1 given back = 3 apples. 

Therefore, Alice would have 3 apples.


### Chain-of-Thought Prompting gpt-3.5

In [12]:
prompt = """Lisa has 7 apples, throws 1 apple, gives 4 apples to Bart and Bart gives one back:
7 - 1 = 6
6 - 4 = 2
2 + 1 = 3

Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

5 - 3 = 2
2 - 2 = 0
0 + 1 = 1

Alice has 1 apple.


* Standard Prompting은 문제를 못풀었지만, Chain-of-Thought Prompting은 문제를 풀었다.

* Standard Prompting gpt-3.5는 문제를 못풀었지만, gpt-4는 문제를 풀 수 있는지 알아보자

### Standard Prompting gpt-4

In [13]:
# Prompt 출처: https://github.com/microsoft/generative-ai-for-beginners/tree/main/05-advanced-prompts

prompt = """Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back, how many apples does Alice have?"""

completion = client.chat.completions.create(
    model="gpt-4-turbo-2024-04-09",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Let's break down the sequence of events to determine how many apples Alice has at the end:

1. Alice starts with 5 apples.
2. Alice throws 3 apples away. Now she has 5 - 3 = 2 apples.
3. Alice gives 2 apples to Bob. Now she has 2 - 2 = 0 apples.
4. Bob gives 1 apple back to Alice. Now she has 0 + 1 = 1 apple.

Therefore, Alice has 1 apple.


* gpt-4는 문제를 풀 수 있다. 
* gpt-4에는 Chain-of-Thought 상태가 녹아있다는 것을 알 수 있다.
* 예전에는 어떤 프롬프트를 쓰면 된다는 점이 있었지만, 요새는 모델이 좋아 좋은 모델을 쓰면 된다.
* 그러나 gpt-4는 gpt-3.5보다 30배 정도 비싸서..ㅎㅎ

## Summary
1. ChatGPT에게 자연스럽게 질문하는 방법 'Zero-Shot Prompting'
2. Zero-Shot보다 더 좋은 방법론은 'Few-Shot', 'Chain-of-Thought'
3. Few-Shot은 Prompt에 예시 추가
4. Chain-of-Thought은 여기에 정답 추리 과정까지 추가
5. Chain-of-Thought은 출력값 Voting 앙상블